In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Request
import numpy as np
import pandas as pd
import requests
import lxml
import line_profiler
import time
import csv
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
recipes = np.loadtxt('recipe_links.csv',dtype=str)
recipe_links = []
for i in range(len(recipes)):
    recipe_links.append(recipes[i].split(','))
recipe_links

[['/food/recipes/aioli_81090'],
 ['/food/recipes/all_about_the_lamb__63859',
  '/food/recipes/ackee_and_saltfish_with_73421',
  '/food/recipes/adhrakigoshtlambmasa_77225',
  '/food/recipes/aged_sirloin_steak_with_62354',
  '/food/recipes/air_fryer_chicken_strips_15576',
  '/food/recipes/air_fryer_chicken_wings_39019',
  '/food/recipes/air_fryer_chipotle_10803',
  '/food/recipes/albanian_baked_lamb_with_92485',
  '/food/recipes/aligot_with_sausages_32912',
  '/food/recipes/aligot_with_roasted_09115'],
 ['/food/recipes/aclassicspongecakewi_9406',
  '/food/recipes/areturntotheblackfor_67199',
  '/food/recipes/aberdeenbutteriesrow_92370',
  '/food/recipes/alefruitloaf_83744'],
 ['/food/recipes/cajun_mayo_55353', '/food/recipes/cajun_seasoning_92363'],
 ['/food/recipes/cabbage_pesto_steamed_99146',
  '/food/recipes/cajun_bbq_burgers_with_09087',
  '/food/recipes/cajun_chicken_wedges_61950',
  '/food/recipes/cajunseabasswithherb_87435',
  '/food/recipes/cajun_seafood_boil_28280',
  '/food/re

In [3]:
recipe_links = [item for sublist in recipe_links for item in sublist]
recipe_links

['/food/recipes/aioli_81090',
 '/food/recipes/all_about_the_lamb__63859',
 '/food/recipes/ackee_and_saltfish_with_73421',
 '/food/recipes/adhrakigoshtlambmasa_77225',
 '/food/recipes/aged_sirloin_steak_with_62354',
 '/food/recipes/air_fryer_chicken_strips_15576',
 '/food/recipes/air_fryer_chicken_wings_39019',
 '/food/recipes/air_fryer_chipotle_10803',
 '/food/recipes/albanian_baked_lamb_with_92485',
 '/food/recipes/aligot_with_sausages_32912',
 '/food/recipes/aligot_with_roasted_09115',
 '/food/recipes/aclassicspongecakewi_9406',
 '/food/recipes/areturntotheblackfor_67199',
 '/food/recipes/aberdeenbutteriesrow_92370',
 '/food/recipes/alefruitloaf_83744',
 '/food/recipes/cajun_mayo_55353',
 '/food/recipes/cajun_seasoning_92363',
 '/food/recipes/cabbage_pesto_steamed_99146',
 '/food/recipes/cajun_bbq_burgers_with_09087',
 '/food/recipes/cajun_chicken_wedges_61950',
 '/food/recipes/cajunseabasswithherb_87435',
 '/food/recipes/cajun_seafood_boil_28280',
 '/food/recipes/cajunspicedsalmonwi

In [4]:
class SpiderSpider(scrapy.Spider):
    name = 'spider'
    handle_httpstatus_list = [404,429]
    
    def errback_httpbin(self, response):
        pass
    
    def __init__(self,links):
        self.allowed_domains = ['https://www.bbc.co.uk']
        self.start_urls=[]
        self.outfile = open("file.csv", "w", newline="")
        self.writer = csv.writer(self.outfile)
        for i in links:
            self.start_urls.append(f"https://www.bbc.co.uk{i}")


    def closed(self,reason):
        self.outfile.close()

        
    def parse(self, response):
        print(response.request.url)
        desc = response.xpath('//a[@class="recipe-ingredients__link"]/@href').getall()
        name =response.xpath('//h1[@class="gel-trafalgar content-title__text"]/text()').getall() 
        print(desc)
        print(name)
        self.writer.writerow(name+desc)
        


In [5]:
%%timeit -n 1 -r 1
from scrapy.crawler import Crawler
if __name__ == "__main__":
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    })
    
    process.crawl(SpiderSpider,links=recipe_links)
    process.start() 
    process.stop()

2022-03-25 23:23:55 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-03-25 23:23:55 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.7.3 (default, Mar 27 2019, 16:54:48) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.3.1, Platform Darwin-18.6.0-x86_64-i386-64bit
2022-03-25 23:23:55 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-03-25 23:23:55 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-03-25 23:23:55 [scrapy.extensions.telnet] INFO: Telnet Password: 73aec4a854ab1aff
2022-03-25 23:23:55 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-03-25 23:

https://www.bbc.co.uk/food/recipes/aioli_81090
['/food/egg_yolk', '/food/garlic', '/food/lemon', '/food/black_pepper', '/food/olive_oil', '/food/mustard_powder', '/food/saffron']
['Aïoli']
https://www.bbc.co.uk/food/recipes/all_about_the_lamb__63859
['/food/olive_oil', '/food/onion', '/food/carrot', '/food/celery', '/food/garlic', '/food/white_wine', '/food/lamb_neck', '/food/chicken_stock', '/food/tomato', '/food/mushroom', '/food/veal', '/food/thyme', '/food/soy_sauce', '/food/olive_oil', '/food/garlic', '/food/courgette', '/food/basil', '/food/chicken_breast', '/food/egg_white', '/food/salt', '/food/white_pepper', '/food/double_cream', '/food/basil', '/food/sweetbread', '/food/haggis', '/food/lamb', '/food/courgette', '/food/courgette', '/food/chervil']
["'All about the lamb' – roast loin of lamb with baby courgettes and basil purée  "]
https://www.bbc.co.uk/food/recipes/ackee_and_saltfish_with_73421
['/food/vegetable_oil', '/food/onion', '/food/garlic', '/food/thyme', '/food/scotch

2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/cabbage_pesto_steamed_99146> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/areturntotheblackfor_67199> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/cajunseabasswithherb_87435> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/cajun_chicken_wedges_61950> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/cajun_bbq_burgers_with_09087> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/albanian_baked_lamb_with_92485> (referer: None)


['/food/mayonnaise', '/food/ketchup', '/food/dijon_mustard']
['Cajun mayo']
https://www.bbc.co.uk/food/recipes/alefruitloaf_83744
['/food/currant', '/food/raisins', '/food/peel', '/food/beer', '/food/brown_sugar', '/food/self-raising_flour', '/food/self-raising_flour', '/food/egg']
['Ale fruit loaf']
https://www.bbc.co.uk/food/recipes/aclassicspongecakewi_9406
['/food/self-raising_flour', '/food/baking_powder', '/food/egg', '/food/butter', '/food/caster_sugar', '/food/vanilla_extract', '/food/icing_sugar', '/food/passion_fruit', '/food/mascarpone_cheese', '/food/caster_sugar', '/food/vanilla_extract', '/food/fromage_frais']
['A classic sponge cake']
https://www.bbc.co.uk/food/recipes/cajun_seasoning_92363
['/food/sea_salt', '/food/paprika', '/food/garlic', '/food/white_pepper', '/food/onion', '/food/cumin', '/food/cayenne_pepper', '/food/oregano', '/food/thyme', '/food/basil']
['Cajun seasoning']
https://www.bbc.co.uk/food/recipes/cabbage_pesto_steamed_99146
['/food/cabbage', '/food/tu

2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/aligot_with_sausages_32912> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/cajun_seafood_boil_28280> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/aberdeenbutteriesrow_92370> (referer: None)
2022-03-25 23:23:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/cajunspicedsalmonwit_84942> (referer: None)


...

https://www.bbc.co.uk/food/recipes/sri_lankan_crab_curry_74562
['/food/plain_flour', '/food/egg', '/food/sugar', '/food/salt', '/food/condensed_milk', '/food/milk', '/food/butter', '/food/sunflower_oil', '/food/crab', '/food/crab', '/food/sunflower_oil', '/food/onion', '/food/garlic', '/food/ginger', '/food/chilli', '/food/cumin', '/food/coriander_seeds', '/food/mustard_seeds', '/food/turmeric', '/food/fenugreek', '/food/curry_leaves', '/food/curry_leaves', '/food/cinnamon', '/food/coconut_milk', '/food/sea_salt', '/food/lime_juice', '/food/coriander_fresh', '/food/coconut']
['Sri Lankan crab curry with Keralan parathas']
https://www.bbc.co.uk/food/recipes/pauls_st_lucia_buns_04825
['/food/milk', '/food/saffron', '/food/butter', '/food/flour', '/food/salt', '/food/yeast', '/food/caster_sugar', '/food/quark', '/food/vegetable_oil', '/food/egg_yolk', '/food/raisins']
['St Lucia buns']
https://www.bbc.co.uk/food/recipes/st_clements_cake_with_96717
['/food/clementine', '/food/ground_almond

2022-03-25 23:27:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/st_clements_orange_and_78030> (referer: None)


https://www.bbc.co.uk/food/recipes/st_clements_orange_and_78030
['/food/caster_sugar', '/food/butter', '/food/lemon', '/food/egg', '/food/self-raising_flour', '/food/egg_yolk', '/food/caster_sugar', '/food/orange', '/food/lemon_juice', '/food/butter', '/food/caster_sugar', '/food/orange', '/food/lemon', '/food/caster_sugar', '/food/icing_sugar', '/food/lemon_juice']
['St. Clements orange and lemon drizzle cake']


2022-03-25 23:27:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bbc.co.uk/food/recipes/squid_ink_pasta_with_03760> (referer: None)
2022-03-25 23:27:08 [scrapy.core.engine] INFO: Closing spider (finished)
2022-03-25 23:27:08 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 1508766,
 'downloader/request_count': 5565,
 'downloader/request_method_count/GET': 5565,
 'downloader/response_bytes': 156701936,
 'downloader/response_count': 5565,
 'downloader/response_status_count/200': 5565,
 'elapsed_time_seconds': 193.057233,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 3, 25, 23, 27, 8, 671027),
 'httpcompression/response_bytes': 652297452,
 'httpcompression/response_count': 5565,
 'log_count/DEBUG': 5566,
 'log_count/INFO': 13,
 'memusage/max': 2279157760,
 'memusage/startup': 117661696,
 'response_received_count': 5565,
 'scheduler/dequeued': 5565,
 'scheduler/dequeued/memory': 5565,
 'scheduler/enqueued': 5565,
 'sche

https://www.bbc.co.uk/food/recipes/squid_ink_pasta_with_03760
['/food/egg_yolk', '/food/egg', '/food/squid', '/food/flour', '/food/olive_oil', '/food/garlic', '/food/spring_onion', '/food/chilli', '/food/ginger', '/food/crab', '/food/white_wine', '/food/parsley', '/food/basil']
['Squid ink pasta with crab and chilli']
3min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
